In [1]:
# step by step guide: https://towardsdatascience.com/multiclass-classification-algorithm-from-scratch-with-a-project-in-python-step-by-step-guide-485a83c79992

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV


In [3]:
test_run = True

In [4]:
df = pd.read_csv("Temp/df_proc_train_set.csv")

In [5]:
df['sol_category'] = pd.to_numeric(df['sol_category'], downcast='integer')
df

,Id,smiles,sol_category,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,EOS12286,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,EOS85869,CCN(CC)[C@H]1CCN(C(=O)Cc2nc(C(C)C)c(C)s2)C1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,EOS85435,CNC(=O)CNC(=O)c1c(-n2cccc2)sc(C)c1C,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,EOS102302,CC(C)(C)c1ccc(CSc2cnn(C(C)(C)C)c(=O)c2Cl)cc1,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EOS64213,CC[C@H](NC(=O)c1ccnc(-n2ccnc2)c1)c1ccccc1OC,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,EOS37839,O=C(NCCCc1nc(=O)[nH][nH]1)[C@H]1CCC(F)(F)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,EOS2088,Cc1ccc(C(=O)NC2CCCC2)cc1S(=O)(=O)N1CCOCC1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,EOS10587,COCCN1CCC(CN(C)S(=O)(=O)c2cccc(C(F)(F)F)c2)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,EOS40533,O=C(Nc1ccc(F)cc1)NC1CCN(C(=O)Cc2cnn(-c3ccccc3)...,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
if test_run:
    df = df.head(5000)
df['sol_category'].value_counts()

1    2835
0    2041
2     124
Name: sol_category, dtype: int64

In [7]:
#y = pd.read_excel(xl, 'y', header = None)

In [8]:
# hypothesis fuction
# takes input variables and theta, return calculated output variable

In [9]:
print(len(df))
#print(len(y))
X = df.copy(deep=True)
X.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X

5000


,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,ecfp_7,ecfp_8,ecfp_9,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4997,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Only exfp

In [10]:
X_ecfp = df.copy(deep=True)
drop_list_desc = [column for column in X_ecfp.columns if (column.startswith('example_') or column.startswith('rdkit_desc_'))]

# Drop features 
X_ecfp.drop(drop_list_desc, axis=1, inplace=True)
X_ecfp.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_ecfp

,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,ecfp_7,ecfp_8,ecfp_9,...,ecfp_4086,ecfp_4087,ecfp_4088,ecfp_4089,ecfp_4090,ecfp_4091,ecfp_4092,ecfp_4093,ecfp_4094,ecfp_4095
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Only descriptors

In [11]:
X_desc = df.copy(deep=True)
drop_list_MFP = [column for column in X_desc.columns if column.startswith('ecfp_')]

# Drop features 
X_desc.drop(drop_list_MFP, axis=1, inplace=True)
X_desc.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_desc

,example_SlogP,example_SMR,example_TPSA,example_NumLipinskiHBA,example_NumLipinskiHBD,example_NumRotatableBonds,example_NumHBA,example_NumAmideBonds,example_NumHeteroAtoms,example_NumAtoms,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,2.60302,103.0537,79.18,8,1,3,6,2,10,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3.06012,92.5500,36.44,4,0,6,4,1,5,51,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.63144,79.5199,63.13,5,2,4,4,2,6,37,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,5.24150,103.2060,34.89,3,0,3,4,0,5,49,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.15700,94.8512,69.04,6,1,6,5,1,6,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3.10070,74.0087,48.99,4,1,2,2,1,7,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,3.20350,85.3440,51.66,5,0,6,5,1,6,40,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4997,2.64240,82.0721,72.19,4,3,3,2,2,5,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,2.28454,87.4057,80.47,5,2,5,4,1,7,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [12]:
y = df.iloc[:, 2]

In [13]:
#y

In [14]:
y.unique()

array([0, 1, 2], dtype=int8)

## StandardScaler

In [15]:
#scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(X)
#X = scaler.fit_transform(X)

## split

In [16]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=1,  shuffle=True, stratify=y)

In [17]:
# find out where NaN is, remove in processing

## multi-class logistic regression

In [18]:
#clf = LogisticRegression(random_state=0, max_iter=1000, verbose=1).fit(X_train, y_train)

In [19]:
#clf.predict(X[:2, :])

In [20]:
#clf.predict_proba(X[:2, :])

In [21]:
#y_prediction = clf.predict(X_test)

In [22]:
#df_y = pd.DataFrame(y_prediction)

In [23]:
# Stricter feature selection
### 1. just use ecfp/rdkit+example
### 2. Hyperparatuning (StratifiedKfold+RandomizedSearchCV on X_train and y_train) (max_iter, solver, penalty,C)

# for dataset over all_dataset
#   hyperparameter tuning()

In [24]:

from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import cohen_kappa_score
from scipy.stats import uniform as sp_randFloat

In [25]:
model = LogisticRegression(random_state=0)

parameters = {
    'max_iter' : range(1000, 2000),
    'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'penalty' : ['l2', 'none'],
    'C' : sp_randFloat()
    }

In [26]:
all_dataset = list()
all_dataset.append(X)
all_dataset.append(X_ecfp)
all_dataset.append(X_desc)

ftwo_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

print('------------------------------------------------------')

for dataset in all_dataset:
    # apply StandardScaler
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(dataset)
    dataset = scaler.fit_transform(dataset)

    # split dataset
    StratifiedKFold(n_splits=5)
    X_train, X_test, y_train, y_test = train_test_split(dataset, y, random_state=1,  shuffle=True, stratify=y)#, test_size=0.2, train_size=0.8)

    # method
    randm_src = RandomizedSearchCV(estimator=model, param_distributions = parameters, cv = 2, n_iter = 10, n_jobs=-1, scoring=ftwo_scorer)
    randm_src.fit(X_train, y_train)

    # output
    print(f'results of data set {dataset}')
    print('------------------------------------------------------')
    print(" Results from Random Search: " )
    print("\n The best estimator across ALL searched params:\n", randm_src.best_estimator_)
    print("\n The best score across ALL searched params:\n", randm_src.best_score_)
    print("\n The best parameters across ALL searched params:\n", randm_src.best_params_)
    print('------------------------------------------------------------------------------------------------------------------------------------------------------------------')



------------------------------------------------------


/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.w

results of data set [[-0.0316386  -0.02829559 -0.19213054 ... -0.24477158 -0.17099726
   2.74189818]
 [-0.0316386  -0.02829559 -0.19213054 ... -0.24477158 -0.17099726
  -0.36049849]
 [-0.0316386  -0.02829559 -0.19213054 ...  3.88988355 -0.17099726
  -0.36049849]
 ...
 [-0.0316386  -0.02829559 -0.19213054 ... -0.24477158 -0.17099726
  -0.36049849]
 [-0.0316386  -0.02829559 -0.19213054 ...  3.88988355 -0.17099726
  -0.36049849]
 [-0.0316386  -0.02829559 -0.19213054 ... -0.24477158 -0.17099726
  -0.36049849]]
------------------------------------------------------
 Results from Random Search: 

 The best estimator across ALL searched params:
 LogisticRegression(C=0.2787101342899051, max_iter=1340, random_state=0)

 The best score across ALL searched params:
 -0.028272132974092257

 The best parameters across ALL searched params:
 {'C': 0.2787101342899051, 'max_iter': 1340, 'penalty': 'l2', 'solver': 'lbfgs'}
----------------------------------------------------------------------------------

/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.w

results of data set [[-0.0316386  -0.02829559 -0.19213054 ... -0.04695584 -0.020004
  -0.09203889]
 [-0.0316386  -0.02829559 -0.19213054 ... -0.04695584 -0.020004
  -0.09203889]
 [-0.0316386  -0.02829559 -0.19213054 ... -0.04695584 -0.020004
  -0.09203889]
 ...
 [-0.0316386  -0.02829559 -0.19213054 ... -0.04695584 -0.020004
  -0.09203889]
 [-0.0316386  -0.02829559 -0.19213054 ... -0.04695584 -0.020004
  -0.09203889]
 [-0.0316386  -0.02829559 -0.19213054 ... -0.04695584 -0.020004
  -0.09203889]]
------------------------------------------------------
 Results from Random Search: 

 The best estimator across ALL searched params:
 LogisticRegression(C=0.9464060192156964, max_iter=1556, random_state=0,
                   solver='newton-cg')

 The best score across ALL searched params:
 -0.02383363095791957

 The best parameters across ALL searched params:
 {'C': 0.9464060192156964, 'max_iter': 1556, 'penalty': 'l2', 'solver': 'newton-cg'}
----------------------------------------------------

/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.w

results of data set [[ 0.15080201  0.41875789  0.2026401  ... -0.24477158 -0.17099726
   2.74189818]
 [ 0.56589791 -0.1124591  -1.59867866 ... -0.24477158 -0.17099726
  -0.36049849]
 [-0.73149699 -0.77144694 -0.47380272 ...  3.88988355 -0.17099726
  -0.36049849]
 ...
 [ 0.18656328 -0.64237128 -0.09196023 ... -0.24477158 -0.17099726
  -0.36049849]
 [-0.13841204 -0.37262834  0.2570084  ...  3.88988355 -0.17099726
  -0.36049849]
 [ 0.07143342  0.86534389  0.31011232 ... -0.24477158 -0.17099726
  -0.36049849]]
------------------------------------------------------
 Results from Random Search: 

 The best estimator across ALL searched params:
 LogisticRegression(C=0.9414665939718218, max_iter=1584, penalty='none',
                   random_state=0)

 The best score across ALL searched params:
 0.011162565667910573

 The best parameters across ALL searched params:
 {'C': 0.9414665939718218, 'max_iter': 1584, 'penalty': 'none', 'solver': 'lbfgs'}
----------------------------------------------

/opt/anaconda3/envs/Kaggle_solubility/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
randm_src

RandomizedSearchCV(cv=2, estimator=LogisticRegression(random_state=0),
                   n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x183206490>,
                                        'max_iter': range(1000, 2000),
                                        'penalty': ['l2', 'none'],
                                        'solver': ['newton-cg', 'lbfgs', 'sag',
                                                   'saga']},
                   scoring=make_scorer(cohen_kappa_score, weights=quadratic))

## Score

In [28]:
#sk_cohen_kappa = cohen_kappa_score(y_prediction, y_test, weights='quadratic')
#print(f'quadratic cohen kappa: {sk_cohen_kappa}')

In [29]:
from sklearn.metrics import confusion_matrix

In [30]:
#confusion_matrix(y_test, y_prediction)

In [31]:
#df_y[0].value_counts()

In [32]:
#len(y_test)